In [5]:
import os
import pandas as pd
import numpy as np
import librosa
import librosa.display
import glob

In [7]:
from sklearn.preprocessing import LabelEncoder
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

/Users/candydong/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [8]:
class Opts():
    def __init__(self):
        self.train_data_path = "./working/train/"
        self.test_data_path = "./working/test/"
        self.csv_path = "./meta"
        self.data_path = "./data/"
        self.model_save_path = "./output/"
        self.model_name = "RF.h5"
        
        if not os.path.exists(self.train_data_path):
            os.makedirs(self.train_data_path)
        if not os.path.exists(self.test_data_path):
            os.makedirs(self.test_data_path)
        if not os.path.exists(self.csv_path):
            os.makedirs(self.csv_path)

In [9]:
opts = Opts()

In [12]:
def df_from_csv(csv_path):
    df = pd.read_csv(csv_path)
    return df

train_csv_path = os.path.join(opts.csv_path, "train.csv")
test_csv_path = os.path.join(opts.csv_path, "test.csv")

train_file_df, test_file_df = df_from_csv(train_csv_path), df_from_csv(test_csv_path)

In [15]:
# use mfcc as feature
def generateFeature(df, opts):
    X, y = [], []
    for idx, (file_path, label) in enumerate(zip(df['file_paths'], df['labels'])):
        s, sr = librosa.load(file_path)
        mfccs = np.mean(librosa.feature.mfcc(y=s, sr=sr, n_mfcc=200).T, axis=0)
        X.append(mfccs)
        y.append(label)
    return X, y

In [17]:
X_train, y_train = generateFeature(train_file_df, opts)
X_test, y_test = generateFeature(test_file_df, opts)

In [18]:
X_train

[array([-3.6065396e+02,  1.0250509e+02, -4.9094658e+01,  2.0267132e+01,
        -3.9502506e+01, -2.0182655e+00, -2.0266191e+01, -5.1506974e-02,
        -1.0015751e+01,  7.2526515e-01, -6.8433456e+00,  4.5786667e+00,
        -7.6762147e+00, -4.7936215e+00, -1.0678170e+01, -2.4660113e+00,
        -7.0902352e+00, -3.2998607e+00, -3.9350901e+00,  6.2522473e+00,
        -3.1997526e+00,  2.0775528e+00, -6.6138515e+00, -6.2004209e-01,
        -5.8491573e+00, -5.3230801e+00, -8.7719975e+00, -3.5580568e+00,
        -7.3242364e+00, -3.8929679e+00, -8.7810392e+00, -4.3526020e+00,
        -7.8662801e+00, -2.6973658e+00, -4.2624536e+00,  5.3281409e-01,
        -4.3035760e+00, -6.0529850e-02, -6.0886278e+00, -2.8429394e+00,
        -3.6238189e+00, -2.5409832e+00, -4.9493308e+00,  1.9674632e-01,
        -5.4101982e+00, -1.3112015e+00, -2.8886170e+00, -5.3053296e-01,
        -2.7731056e+00, -1.0950993e+00, -3.4801836e+00,  5.5029333e-01,
        -2.6537352e+00, -9.9035472e-02, -3.2662241e+00, -1.17372

In [19]:
y_train

[2,
 6,
 0,
 0,
 7,
 7,
 4,
 7,
 5,
 4,
 3,
 7,
 7,
 8,
 7,
 5,
 4,
 8,
 9,
 4,
 2,
 7,
 8,
 7,
 8,
 8,
 9,
 2,
 7,
 3,
 9,
 6,
 8,
 9,
 9,
 5,
 7,
 7,
 8,
 9,
 5,
 4,
 2,
 7,
 4,
 0,
 3,
 7,
 2,
 0,
 3,
 7,
 4,
 2,
 0,
 3,
 0,
 7,
 0,
 0,
 7,
 4,
 3,
 4,
 8,
 7,
 4,
 5,
 3,
 2,
 7,
 7,
 2,
 7,
 8,
 6,
 9,
 7,
 2,
 7,
 4,
 8,
 3,
 7,
 2,
 4,
 7,
 8,
 7,
 5,
 2,
 9,
 3,
 4,
 8,
 7,
 7,
 9,
 4,
 2,
 7,
 7,
 0,
 0,
 0,
 3,
 3,
 0,
 3,
 0,
 7,
 0,
 0,
 3,
 3,
 7,
 2,
 7,
 7,
 9,
 8,
 3,
 5,
 5,
 7,
 4,
 9,
 9,
 3,
 7,
 7,
 3,
 8,
 9,
 9,
 8,
 7,
 4,
 5,
 7,
 5,
 7,
 6,
 7,
 8,
 7,
 3,
 6,
 9,
 0,
 0,
 7,
 1,
 3,
 2,
 6,
 3,
 0,
 7,
 3,
 0,
 6,
 5,
 1,
 3,
 3,
 3,
 2,
 3,
 3,
 7,
 4,
 8,
 8,
 7,
 9,
 9,
 3,
 5,
 7,
 5,
 1,
 1,
 7,
 7,
 8,
 8,
 3,
 9,
 7,
 3,
 7,
 5,
 7,
 2,
 5,
 3,
 8,
 9,
 8,
 9,
 7,
 6,
 6,
 2,
 7,
 3,
 4,
 5,
 0,
 7,
 0,
 4,
 2,
 3,
 0,
 2,
 5,
 7,
 2,
 6,
 5,
 1,
 7,
 0,
 9,
 8,
 5,
 2,
 2,
 9,
 5,
 8,
 2,
 8,
 9,
 6,
 7,
 2,
 7,
 9,
 4,
 9,
 4,
 5,
 0,
 8,
 9,
 4,
 7,


In [29]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [30]:
model.score(X_train, y_train)

0.9988642816581488

In [34]:
import coremltools
mlmodel = coremltools.converters.sklearn.convert(model)
mlmodel.author = "Candy Dong"
mlmodel.input_description["input"] = "an audio clip"
mlmodel.output_description["classLabel"] = "a string value"
mlmodel.short_description = "urban sound classifier"
mlmodel.save(os.path.join(opts.model_save_path,"RF.mlmodel"))